<a href="https://colab.research.google.com/github/EwFox76/Livro-de-receitas-Aula/blob/master/C%C3%B3pia_de_Vers%C3%A3o_estavel02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Instalar bibliotecas necessárias.
!pip install -q google-genai
!pip install -q google-genai ipywidgets
!pip install -q google-adk
print("Bibliotecas instaladas.")

Bibliotecas instaladas.


In [2]:
# Importações Nescessárias.
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
print("API_KEY: OK")

import ipywidgets as widgets
from ipywidgets import HTML, Output, Label, Textarea, Button, VBox, HBox, Layout
from IPython.display import display
from datetime import datetime
print("Widgets: OK")

import google.adk as adk
print("Adk: OK")

from google import genai
client = genai.Client()
model = "gemini-2.5-flash-preview-04-17"
print("Modelo Gemini: OK")

from google.genai import types
print("Tipos para configurar os modelos: OK")



API_KEY: OK
Widgets: OK
Adk: OK
Modelo Gemini: OK
Tipos para configurar os modelos: OK


In [3]:
# Instruções de sistema
chat_config = types.GenerateContentConfig (system_instruction = """
Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto, dicas gerais de bem-estar e positividade para pessoas que podem estar passando por momentos difíceis.
É FUNDAMENTAL que você NUNCA aja como um profissional de saúde mental.
Suas respostas devem ser:
- Empáticas e acolhedoras.
- Focadas em positividade e encorajamento.
- Oferecer dicas GERAIS de bem-estar (ex: sobre sono, alimentação, exercícios leves, hobbies, relaxamento, familiar, lazer, amizade, hidratação, tempo ao ar livre).
- NUNCA fazer diagnósticos ou sugerir tratamentos ou medicamentos.
- SEMPRE incluir um DISCLAIRER CLARO no início ou final de CADA interação de que você não é um profissional e RECOMENDAR FORTEMENTE a busca por AJUDA PROFISSIONAL (psicólogo, psiquiatra).
- NUNCA responder a pedidos de ajuda imediata em crises ou pensamentos suicidas. Nesses casos, instrua a pessoa a procurar ajuda profissional imediatamente, ligar para serviços de emergência ou para um centro de valorização da vida (como o CVV no Brasil, ligando para 188).

Exemplos de coisas para dizer:
- "É corajoso compartilhar como você se sente."
- "Lembre-se de ser gentil consigo mesmo."
- "Pequenos passos fazem diferença."
- "Que tal tentar uma caminhada curta hoje, se possível?"
- "Respirar fundo por alguns minutos pode ajudar a relaxar."
- "Lembre-se que você não está sozinho(a)."
- "Tem pessoas que se importam com você"
- "A vida é um presente único"
- "Beber água e se alimentar bem ajuda no bem-estar geral."

Exemplos de coisas para NÃO dizer:
- "Você tem depressão." (Diagnóstico)
- "Tome este medicamento." (Tratamento)
- "Eu posso curar sua depressão." (Afirmação falsa)
- "Não se sinta assim." (Invalidando sentimentos)
- "É só uma fase, vai passar." (Minimizando)

O DISCLAIMER OBRIGATÓRIO ao final e no início da interação deve ser algo como:
"Olá! Sou um agente de IA de apoio positivo e não substituo a ajuda profissional.
Se você está enfrentando dificuldades significativas, por favor, procure a ajuda de um"
OU
"Lembre-se, sou uma IA e não posso oferecer aconselhamento médico ou psicológico.
Se você precisa de ajuda profissional, por favor, procure um especialista."

OU ALGO SEMELHANTE, MAS CLARO E RECOMENDANDO AJUDA PROFISSIONAL.

Vamos começar. Apresente-se brevemente com o disclaimer inicial.
"""
)
chat = client.chats.create(model=model, config=chat_config)
print("Instruções de sistema: OK")


Instruções de sistema: OK


In [4]:
## Interface Interativa com Widgets

# Variável para controlar se a conversa está ativa
chat_active = True

# Widget de saída para exibir a conversa - Customized Layout and Style
output_area = Output(
    layout={
        'border': '4px solid black',
        'width': 'auto',  # Allow width to adjust
        'height': '300px', # Set a fixed height
        'overflow_y': 'auto' # Add scrollbar if content overflows
    },
    # Using style for background color
    style={
        'background_color': '#f0f0f0' # Light gray background
    }
)

# Widget de entrada para o usuário digitar - Customized Layout
input_area = Textarea(
    placeholder='Digite seus sentimentos aqui...',
    layout=Layout(width='auto', height='100px') # Changed width to auto
)

# Botão para enviar a mensagem - Customized Style
send_button = Button(
    description='Enviar',
    style={
        'button_color': 'darkgreen', # Change button color to light blue
        'text_color': 'black',      # Set text color to black
        'font_weight': 'bold'       # Set font weight to bold
    }
)

# Botão para sair da conversa - Customized Style
exit_button = Button(
    description='Sair da Conversa',
    style={
        'button_color': 'salmon', # Change button color to salmon
        'text_color': 'black',   # Set text color to black
        'font_weight': 'bold'    # Set font weight to bold
    }
)


# Caixa para organizar a entrada e o botão de envio horizontalmente
input_and_send_box = HBox([input_area, send_button]) # Renamed for clarity

# Caixa para organizar os botões de envio e sair na parte inferior
button_box = HBox([send_button, exit_button], layout=Layout(justify_content='space-between')) # New HBox for buttons

# Função para lidar com o clique no botão Enviar
def on_send_button_clicked(b):
    global chat_active
    if chat_active:
        user_input = input_area.value
        input_area.value = '' # Limpa a área de entrada após enviar

        if user_input.lower() == 'sair':
            on_exit_button_clicked(None) # Chama a função de sair se o usuário digitar "sair"
            return

        with output_area:
            print(f"Você: {user_input}\n")
            response = chat.send_message(user_input)
            print(f"Resposta: {response.text}\n")

# Função para lidar com o clique no botão Sair
def on_exit_button_clicked(b):
    global chat_active
    chat_active = False
    input_area.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    exit_button.description = "Saída Solicitada"
    with output_area:
        print("Conversa encerrada. Obrigado por compartilhar!")

# Conectar as funções aos botões
send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Exibir a interface
print("Bem-vindo! Clique no botão 'Sair da Conversa' a qualquer momento para encerrar.")
# Display the widgets in a VBox to stack them vertically
display(output_area, input_area, button_box) # Changed display order and components

# Opcional: Enviar uma mensagem inicial para o chatbot obter a apresentação com disclaimer
# (Isto pode ser adaptado dependendo de como você quer que a conversa inicie)
with output_area:
    # Você pode enviar um prompt inicial "Olá" ou algo similar aqui
    # para fazer o chatbot iniciar a conversa com o disclaimer.
    # Por exemplo:
    response = chat.send_message("Inicie a conversa")
    print(f"Resposta: {response.text}\n")

Bem-vindo! Clique no botão 'Sair da Conversa' a qualquer momento para encerrar.


Output(layout=Layout(border='4px solid black', height='300px', overflow_y='auto', width='auto'))

Textarea(value='', layout=Layout(height='100px', width='auto'), placeholder='Digite seus sentimentos aqui...')